In [1]:
import pandas as pd 
import re
import nltk
import string

pd.set_option('display.max_colwidth',100)
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

df = pd.read_csv('test1_cleaned.csv')
print(df.shape)
df.head()

(32, 3)


,CompanyName,JobDesc,URL
0,NaN,[],https://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D0LburRNBzTqSu7NqtC2F6MbUfWStx2IBZZWMHmcRlz_rjit...
1,Oracle,"['OPERA Certification  PMS\nHotel Systems Project Administration  Onsite Checklists, Project S...",https://indeed.com/rc/clk?jk=c7612ac1efc3f60c&fccid=cd22d01053af7669&vjs=3
2,Woolworths Group,"['New 18 month, Fixed Term opportunity for a Supply Chain Analyst in our Norwest, Bella Vista Of...",https://indeed.com/rc/clk?jk=746567cc51d9ae37&fccid=4fa27cf74ed20337&vjs=3
3,NaN,[],https://indeed.com/rc/clk?jk=13b0ab7adf38f08d&fccid=7ab5f4f72fdce966&vjs=3
4,Verisk Financial,"['Solves challenging business problems and communicates findings to clients, management, and pee...",https://indeed.com/rc/clk?jk=b1874467c3fedfe2&fccid=d621f9a5603cfedd&vjs=3


In [2]:
df = df[df['JobDesc']!='[]']
df

,CompanyName,JobDesc,URL
1,Oracle,"['OPERA Certification  PMS\nHotel Systems Project Administration  Onsite Checklists, Project S...",https://indeed.com/rc/clk?jk=c7612ac1efc3f60c&fccid=cd22d01053af7669&vjs=3
2,Woolworths Group,"['New 18 month, Fixed Term opportunity for a Supply Chain Analyst in our Norwest, Bella Vista Of...",https://indeed.com/rc/clk?jk=746567cc51d9ae37&fccid=4fa27cf74ed20337&vjs=3
4,Verisk Financial,"['Solves challenging business problems and communicates findings to clients, management, and pee...",https://indeed.com/rc/clk?jk=b1874467c3fedfe2&fccid=d621f9a5603cfedd&vjs=3
6,Commonwealth Bank,"['You will successfully undertake analytical deep dives on current portfolio performance, genera...",https://indeed.com/rc/clk?jk=cc47adbcc85dc87d&fccid=26f47cce54403a22&vjs=3
8,KinCare,['Do work that matters  be part of an organisation making a difference in the community\nUnique...,https://indeed.com/rc/clk?jk=d7bbf7a2f3db606f&fccid=18bfccb232e1bc2b&vjs=3
10,KPMG,['\nAre you passionate about Digital Transformation?\nDo you embrace digital disruption?\nAre yo...,https://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DmCA2pcKu53EmPQbuX7PlPZl4HVGQcPQUOWs91zEP-4xo3jo...
12,KPMG,['\nAre you passionate about Digital Transformation?\nDo you embrace digital disruption?\nAre yo...,https://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DmCA2pcKu53EmPQbuX7PlPZl4HVGQcPQUOWs91zEP-4xo3jo...
14,Genworth,['\nTranslate business requirements into clear and measurable outcomes\nProvide detailed analysi...,https://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DveHtaohNZTTr4NJ6iQ6LvBAugsddynFItxJfHbw6F-O3gQv...
15,Allianz,['\nProvide regular analysis on Allianz customers. Develop data visualization of customer experi...,https://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CiRhljGiKDxWVH0BEdBBPtYgp59M8xmYDUCgQPQUjBtAbynH...
16,KUKA Robotics,"['\nDeliver defined parts of a project, including update of specifications, module design, imple...",https://indeed.com/rc/clk?jk=3d9243b7583a2482&fccid=53544a1298fd9768&vjs=3


In [3]:
def clean_text(text):
    text.split('\n')
    text="".join([char.lower() for char in text if char not in string.punctuation])
    tokens = re.split("\W+",text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    return text

df['JobDescCleaned'] = df['JobDesc'].apply(lambda x: clean_text(x))
df.head()

,CompanyName,JobDesc,URL,JobDescCleaned
1,Oracle,"['OPERA Certification  PMS\nHotel Systems Project Administration  Onsite Checklists, Project S...",https://indeed.com/rc/clk?jk=c7612ac1efc3f60c&fccid=cd22d01053af7669&vjs=3,opera certif pmsnhotel system project administr onsit checklist project signoff expens time card...
2,Woolworths Group,"['New 18 month, Fixed Term opportunity for a Supply Chain Analyst in our Norwest, Bella Vista Of...",https://indeed.com/rc/clk?jk=746567cc51d9ae37&fccid=4fa27cf74ed20337&vjs=3,new 18 month fix term opportun suppli chain analyst norwest bella vista officeneffect support as...
4,Verisk Financial,"['Solves challenging business problems and communicates findings to clients, management, and pee...",https://indeed.com/rc/clk?jk=b1874467c3fedfe2&fccid=d621f9a5603cfedd&vjs=3,solv challeng busi problem commun find client manag peersbecom expert data extract analysi lever...
6,Commonwealth Bank,"['You will successfully undertake analytical deep dives on current portfolio performance, genera...",https://indeed.com/rc/clk?jk=cc47adbcc85dc87d&fccid=26f47cce54403a22&vjs=3,success undertak analyt deep dive current portfolio perform gener insight identifi opportun impr...
8,KinCare,['Do work that matters  be part of an organisation making a difference in the community\nUnique...,https://indeed.com/rc/clk?jk=d7bbf7a2f3db606f&fccid=18bfccb232e1bc2b&vjs=3,work matter part organis make differ communitynuniqu opportun drive busi outcom improv across or...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def ngram_vectorize(n):
    ngram_vect = CountVectorizer(ngram_range=(n,n))
    X_counts = ngram_vect.fit_transform(df['JobDescCleaned'])
    X_counts_df = pd.DataFrame(X_counts.toarray())
    X_counts_df.columns = ngram_vect.get_feature_names()
    return X_counts_df

In [5]:
df = ngram_vectorize(3)
df

,18 month fix,23 year experi,3000 store across,3040 time custom,35 year experi,4th industri revolutionnact,abil collabor influenc,abil commun clearli,abil commun present,abil credibl work,...,year experi installingconfiguringtrain,year experi similar,year experi softwar,year experi use,year relat experi,year role relat,year work experi,yet fun environmentnha,zealand woolworth group,zepplin kafka ranger
0,0,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
